This notebook shows how to plot scatter plots, both static and interactive, overlayed on the map of the U.S. It plots predictions for "investment profittability" over 2027-2030 time period, which I am measuring as the rate of change of housing price index (HPI) normalized by 2020 housing value within each 3-digit zipcode. The predictions are made using temperature and precipitation data from IPCC climate model predictions, as outlined in the "Predicting_Future_HPI.ipynb" notebook.

In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import altair as alt
from vega_datasets import data

#Load datasets for state and county boundaries from Altair
states = alt.topo_feature(data.us_10m.url, 'states')
counties = alt.topo_feature(data.us_10m.url, 'counties')

In [7]:
#Make static figures with "profittability" (HPI rate of change normalized by 2020 housing price)
#overlayed on state boundaries map 

#Creat background map with state boundaries
background = alt.Chart(states).mark_geoshape(
    fill='white',
    stroke='black'
).project('albersUsa').properties(
    width=400,
    height=240
)

#Plot "profittability"
#(predicted HPI rate of change divided by 2020 housing price for HPI change based on 2018-2021 data)
scatter_2021 = alt.Chart('predict_df.json',title='Based on 2018-2021 data')\
                .mark_point(filled=True).encode(
                        latitude="INTPTLAT:Q",
                        longitude="INTPTLONG:Q",
                        color=alt.Color(
                            "Most_profittable_2021:Q", scale=alt.Scale(domain=[0,0.25], scheme="tealblues"),
                        )
                ).project('albersUsa')

#Plot "profittability" (predicted HPI rate of change divided by 2020 housing price for predictions using 2027-2030 data)
scatter_2030_2 = alt.Chart('predict_df.json',title='Based on 2027-2030 predictions') \
                .mark_point(filled=True).encode(
                        latitude="INTPTLAT:Q",
                        longitude="INTPTLONG:Q",
                        color=alt.Color(
                            "Most_profittable_2030_8_5:Q", scale=alt.Scale(domain=[0,0.5], scheme="tealblues"),
                            legend=alt.Legend(title="", labelFontSize=15)   )
                        ).project('albersUsa')

#Plot predictions using 2018-2021 data and using 2027-2030 data side-by-side
us_static = (background+scatter_2021 | background+scatter_2030_2).configure_view(stroke=None)
us_static

alt.HConcatChart(...)

In [8]:
#Save profittability predictions (static) in json/html forms
us_static.save('predict_profittable_rcp8_5.json')
us_static.save('predict_profittable_rcp8_5.html')

In [6]:
#Make interactive map with "profittability" (HPI rate of change normalized by 2020 housing price)
#overlayed on state boundaries map where can select regions using drop-down menu

#Creat background map with state boundaries

background = alt.Chart(states).mark_geoshape(
                    fill='white',
                    stroke='black'
             ).project('albersUsa').properties(
                    width=400,
                    height=300
            )

#Plot "profittability" (predicted HPI rate of change divided by 2020 housing price for predictions using 2027-2030 data)
scatter_2030_2 = alt.Chart('predict_df_large.json',title='Regions based on 2027-2030 predictions')\
                    .mark_point(filled=True).encode(
                            latitude="INTPTLAT:Q",
                            longitude="INTPTLONG:Q",
                            color=alt.Color(
                                "Most_profittable_2030_8_5:Q", scale=alt.Scale(domainMid=0, 
                                                                               scheme="redblue", reverse=True),
                                legend=alt.Legend(title="", labelFontSize=15)
                        )
                    ).project('albersUsa')

alt.renderers.enable()

# A dropdown filter
columns = ['All U.S.', 'Great Lakes', 'Rocky Mountain', 'South West', 'Plains', 'South East',
               'New England', 'Mideast', 'Far West']
column_dropdown = alt.binding_select(options=columns,name='Region')
column_select = alt.selection_single(
    fields=['region'],
    on='doubleclick',
    clear=False, 
    bind=column_dropdown, 
    #name='Select',
    init={'region': 'All U.S.'}
)

filter_columns = scatter_2030_2.add_selection(
    column_select
).transform_filter(
    column_select
)

region_interactive = ((background+filter_columns 
                                )).configure_view(stroke=None).properties(padding=10)
region_interactive

alt.LayerChart(...)

In [ ]:
#Save profittability predictions (interactive for regions) in json/html forms
region_interactive.save('predict_regions_rcp8_5.json')
region_interactive.save('predict_regions_rcp8_5.html')

In [9]:
#Read in dictionary that has all the state names and IDs 
import pickle
state_dict = pickle.load(open("state_id_dict.pkl", "rb"))

In [11]:
#Make interactive map with "profittability" (HPI rate of change normalized by 2020 housing price)
#overlayed on state and county boundaries map where can select states using drop-down menu

#Get background counties map: 
#   get "state_id" (which is first two digits of the county id) using transform_calculate
counties_states =(
    alt.Chart(data = counties)
    .mark_geoshape(
        fill='white',
        stroke='black',
        strokeWidth=1,
    ) .transform_calculate(state_id = "(datum.id / 1000)|0"))\
    .project('albersUsa').properties(
        width=400,
        height=300
    )

#Plot "profittability" (predicted HPI rate of change divided by 2020 housing price for predictions using 2027-2030 data)
scatter_2030_2 = alt.Chart('predict_df_all.json',title='Regions based on 2027-2030 predictions')\
                    .mark_point(filled=True).encode(
                            latitude="INTPTLAT:Q",
                            longitude="INTPTLONG:Q",
                            color=alt.Color(
                                "Most_profittable_2030_8_5:Q", scale=alt.Scale(domainMid=0, 
                                                                               scheme="redblue", reverse=True),
                                legend=alt.Legend(title="", labelFontSize=15)
                        )
                    ).project('albersUsa')

#Set-up filter using state id
cols = list(state_dict.values())
cols[0]=None
columns2 = cols
labels = list(state_dict.keys())
column_dropdown2 = alt.binding_select(options=columns2,name='State',labels=labels)
column_select2 = alt.selection_single(
    fields=['state_id'],
    on='doubleclick',
    bind=column_dropdown2, 
)

filter_columns2 = scatter_2030_2.transform_filter(
    column_select2
)

counties_states_select = counties_states.transform_filter(
    column_select2
)

alt.renderers.enable()

#Need to do layer the two charts before adding selection
county_interactive = alt.layer(counties_states_select + filter_columns2).add_selection(
                        column_select2
                    ).configure_view(stroke=None).properties(padding=10)

county_interactive.display()


alt.LayerChart(...)

In [ ]:
#Save profittability predictions (interactive for states with county boundaries) in json/html forms
county_interactive.save('predict_states_counties_rcp8_5.json')
county_interactive.save('predict_states_counties_rcp8_5.html')